In [1]:
from src.utils import read_results, show_table_mean_std

root_dir = "./"
experiment = "gpt2-xl_n100"
# experiment = "cb"
# experiment = "gpt3-text-davinci-003_n100"

!python collect_results.py --root_dir $root_dir --experiment $experiment --config noneboots_acc

Done!                                                                           


In [2]:
prob_types = None
# prob_types = ["original_probs", "content_free_['idk']"]

# dataset = ["cb", "trec"]
dataset = ["cb", "rte", "sst2", "trec", "agnews"]

results, score, model_name = read_results(root_dir, experiment)
show_table_mean_std(results, score, model_name, prob_types=prob_types, dataset=dataset)